In [1]:
import pandas as pd
import numpy as np
import re
import os
import nltk
import pymorphy2
from os.path import join
from sklearn.model_selection import StratifiedShuffleSplit

## Загрузка; преобразование; чистка;

In [2]:
# загрузка csv файла из датасета
dataset_dir = './data/'
dataset_names = os.listdir(dataset_dir)
print('Avaliable datasets: {}'.format(dataset_names))

files = os.listdir(join(dataset_dir, dataset_names[0]))
print('Avaliable files in dataset {0}: \n{1}'.format(dataset_names[0], files))

Avaliable datasets: ['vkusvill']
Avaliable files in dataset vkusvill: 
['07.xls', '17.xls', '03.xls', '11.xls', '15.xls', 'X_test.csv', '08.xls', 'Тип жалобы.xlsx', '04.xls', 'X_train.csv', '12.xls', '14.xls', '10.xls', '02.xls', '01.xls', 'vkusvill_all_categories.csv', '05.xls', 'data_exploration_report.html', '09.xls']


In [20]:
def read_dataset(filepath, duplicates=False, clean=True):
    
    file = open(filepath, 'r', encoding='ISO-8859-1')
    data = pd.read_csv(file)
    
    old_names = data.keys()
    names = [n.encode('ISO-8859-1').decode('cp1251').encode('utf8') for n in old_names]
    names = [n.decode('utf-8') for n in names]
    
    new_data = dict()
    for old, new in zip(old_names, names):
        new_data[new] = list()
        for c in data[old]:
            try:
                s = c.encode('ISO-8859-1').decode('cp1251').encode('utf8')
                s = s.decode('utf-8')
                new_data[new].append(s)
            except AttributeError:
                new_data[new].append(c)
    
    new_data = pd.DataFrame(new_data, columns=['Описание', 'Категория жалобы'])
    new_data.rename(columns={'Описание': 'req', 'Категория жалобы': 'cat'}, inplace=True)
    new_data = new_data.dropna()  # dell nan
    if not duplicates:
        new_data = new_data.drop_duplicates()  # dell duplicates
    
    # как отдельную ветвь можно использовать
    if clean:
        delete_bad_symbols = lambda x: " ".join(re.sub('[^а-яa-zё0-9]', ' ', x.lower()).split())
        new_data['req'] = new_data['req'].apply(delete_bad_symbols)
    
    new_data = new_data.reset_index()
    
    return new_data

def read_dataset_fromV(filepath, duplicates=False, clean=True):
    file = open(filepath, 'r')
    data = pd.read_csv(file)

    new_data = pd.DataFrame(data, columns=['Описание', 'Категория жалобы'])
    new_data.rename(columns={'Описание': 'req', 'Категория жалобы': 'cat'}, inplace=True)
    new_data = new_data.dropna()  # dell nan
    if not duplicates:
        new_data = new_data.drop_duplicates()  # dell duplicates

    # как отдельную ветвь можно использовать
    if clean:
        delete_bad_symbols = lambda x: " ".join(re.sub('[^а-яa-zё0-9]', ' ', x.lower()).split())
        new_data['req'] = new_data['req'].apply(delete_bad_symbols)

    new_data = new_data.reset_index()

    return new_data

In [21]:
path = join(dataset_dir, dataset_names[0], 'X_train.csv')
# data = read_dataset(path, False)
# data = pd.read_csv(path)
data = read_dataset_fromV(path, False)
print(data.keys())



/home/mks/intent/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (6,7,8,9,10,11,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['index', 'req', 'cat'], dtype='object')


In [23]:
print(data['cat'])

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
29665    15
29666    16
29667    16
29668    16
29669    16
29670    16
29671    16
29672    16
29673    16
29674    16
29675    16
29676    16
29677    16
29678    16
29679    16
29680    16
29681    16
29682    16
29683    16
29684    16
29685    16
29686    16
29687    16
29688    16
29689    16
29690    16
29691    16
29692    16
29693    16
29694    16
Name: cat, Length: 29695, dtype: int64


In [10]:
k = 0
index = []
for i in range(len(data['cat'])):
    try:
        v = data['cat'][i]
    except KeyError:
        index.append(i)
        k += 1

print(k)
print(index)

0
[]


## Датасет теперь не содержит пропущенных значений, пустых строк, или запросов без указания к какому класса они относятся. Однако есть повторяющиеся значения:

In [11]:
data['req'].value_counts().head(5)

активация мтс            20
активация                14
вопросы по лп             9
лп                        9
привязали чек к карте     7
Name: req, dtype: int64

## Повторяются чаще всего короткие обращения всего в несколько слов.

In [10]:
print('Всего запросов: {}'.format(len(data['req'])))
print('Количество запросов после сортировки: {}'.format(len(data['req'].unique())))
print('Количество повторяющихся запросов: {}'.format(len(data['req']) - len(data['req'].unique())))

Всего запросов: 58780
Количество запросов после сортировки: 44652
Количество повторяющихся запросов: 14128


In [11]:
path = join(dataset_dir, dataset_names[0], 'vkusvill_all_categories.csv')
data = read_dataset(path)
print(data.keys())

data['req'].value_counts().head(5)

/home/mks/intent/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['req', 'cat'], dtype='object')


активация мтс            20
активация                14
вопросы по лп             9
лп                        9
привязали чек к карте     7
Name: req, dtype: int64

# Препроцессинг

## Преобразование классов. Создание класса "Others" и перенос в него соответствующих данных по значению порога

In [305]:
def other_(data):
    
    clas = data['Категория жалобы'].unique()
    cld = dict()
    cll = list()
    
    for x in clas:
        cld[x] = data[data['Категория жалобы'] == x]['Категория жалобы'].value_counts()
        cll.append(data[data['Категория жалобы'] == x]['Категория жалобы'].value_counts())
    
    cll = np.array(cll)
    mean = cll.mean()
    
    new_cl = 0
    k = 1
    for i, x in enumerate(clas):
        if cld[x] <= 0.5*mean:  # rule for "others" class
            new_cl += cld[x]
            cld[x] = 0
        else:
            cld[x] = k + 1
    
    for x in clas:
        data[data['Категория жалобы'] == x]['Категория жалобы'] = cld[x]
    
    print('Request in others: {}'.format(new_cl))
    return data


def other(data, threshold='half_mean'):
    
    clas = data['Категория жалобы'].unique()
    cld = dict()
    cll = list()
    
    for x in clas:
        d = data[data['Категория жалобы'] == x]['Категория жалобы'].value_counts()
        c = int(list(d.items())[0][1])
        cld[x] = c
        cll.append(c)
    
    cll = np.array(cll)
    mean = cll.mean()
    print(mean)
    
    new_cl = 0
    k = 1
    for i, x in enumerate(clas):
        if cld[x] <= 0.5*mean:  # rule for "others" class
            new_cl += cld[x]
            cld[x] = 0
        else:
            cld[x] = k + 1
    
    for x in clas:
        data['Категория жалобы'] = cld[x]
    
    print('Request in others: {}'.format(new_cl))
    return data

In [12]:
classes = data['cat'].unique()

for i in range(len(classes)):
    s = data[data['cat'] == classes[i]]['cat'].value_counts()
    print('*** {} {} ***'.format(i, classes[i - 1]))
    print('\n'.join(['{:<50} {}'.format(x[0], x[1]) for x in list(s.items())]))
    print('-' * 100)

*** 0 17 ***
1                                                  3288
----------------------------------------------------------------------------------------------------
*** 1 1 ***
2                                                  5312
----------------------------------------------------------------------------------------------------
*** 2 2 ***
3                                                  6791
----------------------------------------------------------------------------------------------------
*** 3 3 ***
4                                                  1366
----------------------------------------------------------------------------------------------------
*** 4 4 ***
5                                                  80
----------------------------------------------------------------------------------------------------
*** 5 5 ***
6                                                  9600
----------------------------------------------------------------------------------------

In [306]:
data_ = other(data)

print(data_['cat'].unique())

2657.1176470588234


/home/mks/intent/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Request in others: 3785
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


# Обработка текста

## Токенизация

In [46]:
# nltk.download('punkt')
sent = data['req'][5]
tags = nltk.word_tokenize(sent)
print(sent)
print(tags)


день добрый всегда был доволен качеством ваших продуктов берем всей семьей и друзьями рекомендовали 13 08 2016 в магазине вкусвил на бульваре яна райниса д 2 корп 1 закупился продуктами карты 1713654 в числе которых был творог 9 сегодня открыл и попробовал его и поранился осколками стекла которые были в нем кусок стекла который вытащил могу при необходимости сфотографировать и отправить не знаю что еще написать с уважением антон близнюк skype antonith моб 89060482086 сегодня вернулся с работы и обнаружил что мои выбросили упаковку просто возьмите на заметку
['день', 'добрый', 'всегда', 'был', 'доволен', 'качеством', 'ваших', 'продуктов', 'берем', 'всей', 'семьей', 'и', 'друзьями', 'рекомендовали', '13', '08', '2016', 'в', 'магазине', 'вкусвил', 'на', 'бульваре', 'яна', 'райниса', 'д', '2', 'корп', '1', 'закупился', 'продуктами', 'карты', '1713654', 'в', 'числе', 'которых', 'был', 'творог', '9', 'сегодня', 'открыл', 'и', 'попробовал', 'его', 'и', 'поранился', 'осколками', 'стекла', 'кот

In [47]:
sent = data['req'][5]
sentences = nltk.sent_tokenize(sent)

print(sent)
print(sentences)

for s in sentences:
    tags = nltk.word_tokenize(s)
    print(tags)

день добрый всегда был доволен качеством ваших продуктов берем всей семьей и друзьями рекомендовали 13 08 2016 в магазине вкусвил на бульваре яна райниса д 2 корп 1 закупился продуктами карты 1713654 в числе которых был творог 9 сегодня открыл и попробовал его и поранился осколками стекла которые были в нем кусок стекла который вытащил могу при необходимости сфотографировать и отправить не знаю что еще написать с уважением антон близнюк skype antonith моб 89060482086 сегодня вернулся с работы и обнаружил что мои выбросили упаковку просто возьмите на заметку
['день добрый всегда был доволен качеством ваших продуктов берем всей семьей и друзьями рекомендовали 13 08 2016 в магазине вкусвил на бульваре яна райниса д 2 корп 1 закупился продуктами карты 1713654 в числе которых был творог 9 сегодня открыл и попробовал его и поранился осколками стекла которые были в нем кусок стекла который вытащил могу при необходимости сфотографировать и отправить не знаю что еще написать с уважением антон б

In [21]:
morph = pymorphy2.MorphAnalyzer()


def tokenize(sen):
    sent_toks = nltk.sent_tokenize(sen)
    word_toks = [nltk.word_tokenize(el) for el in sent_toks]
    tokens = [val for sublist in word_toks for val in sublist]
    tokens = [el for el in tokens if el != '']
    tokens = [el.lower() for el in tokens]
    tokens = [morph.parse(el)[0].normal_form for el in tokens]
    return tokens


In [23]:
Tokens = tokenize(data['req'][5])


In [ ]:
print(Tokens)


## Векторизация

In [28]:
from fasttext import load_model
emb_dict = load_model('./embeddings/russian/ft_0.8.3_nltk_yalen_sg_300.bin')

In [ ]:
emb = np.zeros((len(Tokens), 300))
for i, x in enumerate(Tokens):
    emb[i] = emb_dict[x]

print(emb)


['Ну что жe.', 'Настало время проверить всё ли нормально.']
['Настало', 'время', 'проверить', 'всё', 'ли', 'нормально', '.']
['ну', 'что', 'жe', '.', 'настать', 'время', 'проверить', 'весь', 'ли', 'нормальный', '.']


In [51]:
s = 'Ну что жe. Настало время проверить всё ли нормально.'
sent = nltk.sent_tokenize(s)
print(sent)
for x in sent:
    tok = nltk.word_tokenize(x)
print(tok)

tok_ = tokenize(s)
print(tok_)

['Ну что жe.', 'Настало время проверить всё ли нормально.']
['Настало', 'время', 'проверить', 'всё', 'ли', 'нормально', '.']
['ну', 'что', 'жe', '.', 'настать', 'время', 'проверить', 'весь', 'ли', 'нормальный', '.']


In [64]:
from tqdm import tqdm


def tokenization(data, morph=True):
    dataset = list()
    ans = list()
    
    for x, y in tqdm(zip(data['req'], data['cat'])):
        if morph:
            tokens = tokenize(x)
        else:
            sent = nltk.sent_tokenize(x)
            word_toks = [nltk.word_tokenize(el) for el in sent]
            tokens = [val for sublist in word_toks for val in sublist]
        
        dataset.append(tokens)
        ans.append(y)    
    
    return dataset, ans

TypeError: only integer scalar arrays can be converted to a scalar index

In [65]:
x, y = tokenization(data)

TypeError: only integer scalar arrays can be converted to a scalar index

In [69]:
print(x[0:5])


TypeError: only integer scalar arrays can be converted to a scalar index

In [73]:
# разбиваем датасет на train/test с сохранением соотношений
n = 2
skf = StratifiedShuffleSplit(n_splits=n, test_size=0.2, random_state=0)
# skf.get_n_splits(x, y)

dataset = list()
for train_index, test_index in skf.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    dataset.append(((X_train, Y_train), (X_test, Y_test)))

TypeError: only integer scalar arrays can be converted to a scalar index

In [74]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
X, Y = make_classification(n_features=4, random_state=0)
clf = LinearSVC(random_state=0)

In [76]:
X.shape

(100, 4)

In [5]:
prec = [0.9, 0.75, 0.63, 0.5, 0.001, 0.82, 0.73, 0.51, 0.7, 0.75, 0.7, 0.68, 0.75, 0.99, 0.92, 0.8]
recall = [0.8, 0.8, 0.77, 0.08, 0.001, 0.9, 0.8, 0.2, 0.42, 0.78, 0.58, 0.61, 0.75, 0.02, 0.78, 0.9]

prec = np.array(prec)
rec = np.array(recall)

f1_ = 2 * (prec * rec)/(prec + rec)
print('F1 massive:\n {}'.format(f1_))

f1 = np.mean(f1_)
print('F1 macro: {}'.format(f1))

F1 massive:
 [0.84705882 0.77419355 0.693      0.13793103 0.001      0.85813953
 0.76339869 0.28732394 0.525      0.76470588 0.634375   0.64310078
 0.75       0.03920792 0.84423529 0.84705882]
F1 macro: 0.5881080796088309


In [24]:
scoring_dict = dict()

for x in data['cat'].unique():
    scoring_dict[x] = list()
    
for i in range(len(data)):
    scoring_dict[data['cat'][i]].append((data['req'][i], data['cat'][i]))

In [33]:
print(len(scoring_dict[1]))

3288


In [ ]:
z = np.zeros((5, 5))
n = np.ones_like(z) * 5
print(z)
print(n)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5.]
 [5. 5. 5. 5. 5.]]


In [ ]:
1000